In [ ]:
%pip install openai

In [1]:
import json
from openai import OpenAI

In [2]:
client = OpenAI(api_key="sk-32e5e7a6f0a74aea99fc06a0b0a670f4", base_url="https://api.deepseek.com")

In [3]:
raw_book = ""
with open('killers.txt', 'r', encoding='utf-8') as file:
    line_number = 1
    for line in file:
        raw_book += f"{line_number}: {line}"
        line_number += 1

gold = ""
with open('killers_gold_tags.txt', 'r', encoding='utf-8') as file:
    line_number = 1
    for line in file:
        gold += f"{line_number}: {line}"
        line_number += 1

golden_plate = ""
with open('goldenplate.txt', 'r', encoding='utf-8') as file:
    for line in file:
        golden_plate += f"{line}"



In [6]:
example = """[
  {
      "name": "Harry",
    "connections": [
      { "target": "Jack", "type": "friend", "strength": 0.8 },
      { "target": "Louise", "type": "colleague", "strength": 0.4 },
      { "target": "Station", "type": "place", "strength": 0.4 }
    ]
  },
  {
    "name": "Jack",
    "connections": [
      { "target": "Jack", "type": "friend", "strength": 0.8 },
      { "target": "Louise", "type": "friend", "strength": 0.9 }
    ]
  },
  {
    "name": "Louise",
    "connections": [
      { "target": "Jack", "type": "friend", "strength": 0.8 },
      { "target": "Harry", "type": "colleague", "strength": 0.9 },
    ]
  }
]"""

system_prompt = f'You are an expert annotator of literary works. For a given text, generate a character network including all the characters in the text. Make sure to include characters from all the chapters and parts of the text. Also record the relations between characters and the strength (i.e. the frequency and intensity) of the interactions. Use the following JSON template to generate the network:' + example

prompt = "Extract all the characters (persons) from the following text. If one character occurs under several identities, only include the first identity, but include the other identities as aliases. Output the craracters in the following json format: [{'description': 'character's description', 'name': 'character's main name', 'aliases' ['alias1','alias2'...]}]\n" + golden_plate
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        #{"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ],
    response_format={
        'type': 'json_object'
    },
    temperature=0,max_tokens=8000,
    stream=False
)

answer = response.choices[0].message.content

In [ ]:
answer = answer

In [31]:
def tag_lines_iterative(dr_data, text, generated_output):
    # TODO: pick characters from dr_data
    example = '{1: {"sentence": "“How is it going” George asked them.", "tags": {"speakers": ["George"], "addressees": ["Al","Max"],"mentioned_characters": [], "other_present_characters": ["Nick"], "place": "Lunchroom"}, 2: {"sentence": "Mrs. Bell washing a shirt that belonged to the man.", "tags": {"speakers": [], "addressees": [],"mentioned_characters": ["Mrs. Bell"],"indirectly_mentioned_characters": ["Ole"], "other_present_characters": [], "place": "Lunchroom"}}\n'

    system_prompt = "You are a an expert annotator of literary text. The user will provide a literary work, with each sentence on its own line and prefixed with a line number. You will annotate each sentence of the work with tags corresponding to the characters and places that are relevant to the sentence. The user will provide a list of valid tags. Annotate the following information: 1. If the sentence contains a line of dialogue, annotate the tag for the speaker and the tag for the character(s) being addressed. 2. If the sentence mentions a character or characters directly by name, annotate the character's tag. 3. If the sentence refers to characters  indirectly e.g. by using a pronoun, such as 'he' or 'they', or a description, such as 'the man' or 'two women', annotate the characters' tags. If it's not clear who the indirect mention refers to, check it from the textual context before and after the sentence. 4. If you can deduct from the context of the sentence, that other characters are present in the situation described by the sentence, also annotate those characters' tags. Make sure to include all relevant tags. The annotations should be recorded in JSON, and you should only output JSON without any explanation or description. When the JSON output is complete, output a line containing 'END OF ANALYSIS' to signal that the analysis is complete. Use the following JSON format:"+example

    prompt = f'Annotate the following text. Valid character tags are {", ".join(dr_data["characters"][:-1])}, and {dr_data["characters"][-1]}. Valid place tags are {", ".join(dr_data["places"][:-1])}, and {dr_data["places"][-1]}\n' + text + "\n"

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": generated_output, "prefix": True}
        ],
        temperature=0,max_tokens=8000,
        stream=False
    )

    return response.choices[0].message.content
    

In [32]:
dr_data = {
    "characters": ["Max", "Al", "George", "Nick", "Sam", "Ole", "Mrs. Bell", "Motorman"], 
    "places": ["Lunchroom", "Boarding house"]
    }
all_output = ""
short_book = "\n".join(raw_book.split("\n")[0:50])
#TODO: Indirect refs to Sam still fail sometimes.
while True:
    output = tag_lines_iterative(dr_data,raw_book,all_output)
    if not output:
        break
    all_output += output
print(all_output)

```json
{
  "1": {
    "sentence": "The door of Henry’s lunchroom opened and two men came in.",
    "tags": {
      "speakers": [],
      "addressees": [],
      "mentioned_characters": [],
      "indirectly_mentioned_characters": ["Max", "Al"],
      "other_present_characters": ["George", "Nick"],
      "place": "Lunchroom"
    }
  },
  "2": {
    "sentence": "They sat down at the counter.",
    "tags": {
      "speakers": [],
      "addressees": [],
      "mentioned_characters": [],
      "indirectly_mentioned_characters": ["Max", "Al"],
      "other_present_characters": ["George", "Nick"],
      "place": "Lunchroom"
    }
  },
  "3": {
    "sentence": "“What’s yours?” George asked them.",
    "tags": {
      "speakers": ["George"],
      "addressees": ["Max", "Al"],
      "mentioned_characters": ["George"],
      "indirectly_mentioned_characters": [],
      "other_present_characters": ["Nick"],
      "place": "Lunchroom"
    }
  },
  "4": {
    "sentence": "“I don’t know,” one of th

In [32]:
print(json.dumps(output,ensure_ascii=False)[:-1])

{"1": {"sentence": "The door of Henry’s lunchroom opened and two men came in.", "tags": {"speakers": [], "addressees": [], "other_present_characters": ["Al", "Max"], "place": "Lunchroom"}}, "2": {"sentence": "They sat down at the counter.", "tags": {"speakers": [], "addressees": [], "other_present_characters": ["Al", "Max"], "place": "Lunchroom"}}, "3": {"sentence": "“What’s yours?” George asked them.", "tags": {"speakers": ["George line"], "addressees": ["Al", "Max"], "other_present_characters": ["Nick"], "place": "Lunchroom"}}, "4": {"sentence": "“I don’t know,” one of the men said.", "tags": {"speakers": ["Al line"], "addressees": ["George"], "other_present_characters": ["Max", "Nick"], "place": "Lunchroom"}}, "5": {"sentence": "“What do you want to eat, Al?”", "tags": {"speakers": ["Max line"], "addressees": ["Al"], "other_present_characters": ["George", "Nick"], "place": "Lunchroom"}}, "6": {"sentence": "“I don’t know,” said Al.", "tags": {"speakers": ["Al line"], "addressees": ["

In [23]:
print(tag_json)

{'1': {'sentence': 'The door of Henry’s lunchroom opened and two men came in.', 'tags': ['Lunchroom', 'Al', 'Max']}, '2': {'sentence': 'They sat down at the counter.', 'tags': ['Al', 'Max', 'Lunchroom']}, '3': {'sentence': '“What’s yours?” George asked them.', 'tags': ['George', 'Al', 'Max', 'George line', 'Lunchroom']}, '4': {'sentence': '“I don’t know,” one of the men said.', 'tags': ['Al', 'Max', 'Lunchroom']}, '5': {'sentence': '“What do you want to eat, Al?”', 'tags': ['Max', 'Al', 'Max line', 'Lunchroom']}, '6': {'sentence': '“I don’t know,” said Al.', 'tags': ['Al', 'Al line', 'Lunchroom']}, '7': {'sentence': '“I don’t know what I want to eat.”', 'tags': ['Al', 'Al line', 'Lunchroom']}, '8': {'sentence': 'Outside it was getting dark.', 'tags': ['Lunchroom']}, '9': {'sentence': 'The streetlight came on outside the window.', 'tags': ['Lunchroom']}, '10': {'sentence': 'The two men at the counter read the menu.', 'tags': ['Al', 'Max', 'Lunchroom']}, '11': {'sentence': 'From the othe

In [7]:
def compare_letter_sequence(str1, str2):
    """
    Compare two strings to see if they have the same letters in the same order,
    ignoring all non-letter characters (using isalpha()).
    
    Args:
        str1 (str): First string to compare
        str2 (str): Second string to compare
        
    Returns:
        bool: True if the letter sequences match, False otherwise
    """
    # Extract letters from each string
    letters1 = [c.lower() for c in str1 if c.isalpha()]
    letters2 = [c.lower() for c in str2 if c.isalpha()]
    
    return letters1 == letters2

# Example usage:
print(compare_letter_sequence("Hello, World!", "h e l l o w o r l d"))  # True
print(compare_letter_sequence("Python3", "P y t h o n"))  # True
print(compare_letter_sequence("abc", "a b c d"))  # False
print(compare_letter_sequence("Test", "t e s t"))  # True
print(compare_letter_sequence("Case", "c a s e"))  # True
print(compare_letter_sequence("Different", "Letters"))  # False

True
True
False
True
True
False


In [11]:
def calculate_f1_score(test_list, gold_list):
    test_set = set(test_list)
    gold_set = set(gold_list)

    true_positives = len(test_set & gold_set)
    predicted_positives = len(test_set)
    actual_positives = len(gold_set)

    precision = true_positives / predicted_positives if predicted_positives > 0 else 0
    recall = true_positives / actual_positives if actual_positives > 0 else 0

    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2 * precision * recall / (precision + recall)

    return precision, recall, f1

def macro_f1(test_lists, gold_lists):
    f1s = []
    for test, gold in zip(test_lists, gold_lists):
        _, _, f1 = calculate_f1_score(test, gold)
        f1s.append(f1)
    return sum(f1s) / len(f1s) if f1s else 0

def micro_f1(test_lists, gold_lists):
    total_tp = total_pred = total_gold = 0
    for test, gold in zip(test_lists, gold_lists):
        test_set = set(test)
        gold_set = set(gold)
        total_tp += len(test_set & gold_set)
        total_pred += len(test_set)
        total_gold += len(gold_set)
    
    precision = total_tp / total_pred if total_pred > 0 else 0
    recall = total_tp / total_gold if total_gold > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    return f1


In [26]:
import ast
tags = [set(item["tags"]) for item in tag_json.values()]
print(tags)
gold_tags = [set(ast.literal_eval(x.split(" ||| ")[1])) for x in gold.split("\n")]
print(gold_tags)
print(set(gold_tags[0]))
for (sentence,sent_tags,sent_gold_tags) in zip([f'{item["sentence"]}' for item in tag_json.values()],tags,gold_tags):
    #print(sent_tags)
    #print(sent_gold_tags)
    too_many_tags = set(sent_tags) - set(sent_gold_tags)
    not_all_tags = set(sent_gold_tags) - set(sent_tags)
    print(f"{sentence}: extra tags: {too_many_tags}. missing tags: {not_all_tags} gold tag: {sent_gold_tags}")
print(macro_f1(tags,gold_tags))



[{'Al', 'Max', 'Lunchroom'}, {'Al', 'Max', 'Lunchroom'}, {'Lunchroom', 'Al', 'George line', 'Max', 'George'}, {'Al', 'Max', 'Lunchroom'}, {'Max', 'Lunchroom', 'Al', 'Max line'}, {'Al', 'Al line', 'Lunchroom'}, {'Al', 'Al line', 'Lunchroom'}, {'Lunchroom'}, {'Lunchroom'}, {'Al', 'Max', 'Lunchroom'}, {'Al', 'Nick', 'Max', 'Lunchroom'}, {'Nick', 'Lunchroom', 'George'}, {'Max', 'Lunchroom', 'Max line'}, {'Lunchroom', 'George line', 'George'}, {'Max', 'Lunchroom', 'Max line'}, {'Lunchroom', 'George line', 'George'}, {'Lunchroom', 'George line', 'George'}, {'Lunchroom', 'George'}, {'Lunchroom', 'George line', 'George'}, {'Al', 'Al line', 'Lunchroom'}, {'Lunchroom', 'George line', 'George'}, {'Max', 'Lunchroom', 'Max line'}, {'Max', 'Lunchroom', 'Max line'}, {'Lunchroom', 'George line', 'George'}, {'Lunchroom', 'George line', 'George'}, {'Max', 'Lunchroom', 'Max line'}, {'Lunchroom', 'George line', 'George'}, {'Max', 'Lunchroom', 'Max line'}, {'Max', 'Lunchroom', 'Max line'}, {'Lunchroom', 'G

In [20]:
# Extract sentences
sentences = [f'{item["sentence"]} ||| {item["tags"]}' for item in tag_json.values()]


print(tag_json)
# Print the sentences
for sentence in sentences:
    print(sentence)

print(len(sentences))
for line in raw_book.split("\n"):
    if sentences and compare_letter_sequence(line,sentences[0]):
        sentences.pop(0)
print(len(sentences))


{'1': {'sentence': 'The door of Henry’s lunchroom opened and two men came in.', 'tags': ['Lunchroom', 'Max', 'Al']}, '2': {'sentence': 'They sat down at the counter.', 'tags': ['Lunchroom', 'Max', 'Al']}, '3': {'sentence': '“What’s yours?” George asked them.', 'tags': ['George line', 'Max', 'Al']}, '4': {'sentence': '“I don’t know,” one of the men said.', 'tags': ['Max line', 'Al']}, '5': {'sentence': '“What do you want to eat, Al?”', 'tags': ['Max line', 'Al']}, '6': {'sentence': '“I don’t know,” said Al.', 'tags': ['Al line', 'Max']}, '7': {'sentence': '“I don’t know what I want to eat.”', 'tags': ['Al line', 'Max']}, '8': {'sentence': 'Outside it was getting dark.', 'tags': ['Lunchroom']}, '9': {'sentence': 'The streetlight came on outside the window.', 'tags': ['Lunchroom']}, '10': {'sentence': 'The two men at the counter read the menu.', 'tags': ['Lunchroom', 'Max', 'Al']}, '11': {'sentence': 'From the other end of the counter Nick Adams watched them.', 'tags': ['Lunchroom', 'Nick